In [2]:
import os
import ipywidgets as widgets

from comet_ml import API

from ift6758.client import ServingClient
from ift6758.game_client import GameClient

In [3]:
COMET_API_KEY = os.environ.get("COMET_API_KEY")
# COMET_API_KEY = "hdTbgQGSQDtOoxv7ZbHrOx2OU"
api = API(api_key=COMET_API_KEY)

In [5]:
workspace_value = os.environ.get("COMET_DEFAUTL_MODEL_WORKSPACE", 'jaihon')
workspace = widgets.Dropdown(
    options=[workspace_value],
    value=workspace_value,
    description='Workspace:',
    disabled=False,
)

models_options = api.get_registry_model_names(workspace.value)[1:]
model = widgets.Dropdown(
    options=models_options,
    value=models_options[2],
    description='Model:',
    disabled=False,

)

version_options = api.get_registry_model_versions(workspace.value, model.value)
versions = widgets.Text(
    placeholder='Version model',
    description='Version:',
    disabled=False,

)

download_button = widgets.Button(
    description='Download',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Downloads the model',
    icon='download', # (FontAwesome names without the `fa-` prefix)

)

In [6]:
models_widgets = widgets.HBox([workspace, model, versions, download_button])

In [7]:
game_id_input = versions = widgets.Text(
    placeholder='game ID',
    description='Game ID:',
    disabled=False,
)
upload_game_info = widgets.Button(
    description='Update Predictions',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Fetch new game information and update predictions',
    icon='refresh', # (FontAwesome names without the `fa-` prefix)
)

In [8]:
game_widgets = widgets.HBox([game_id_input, upload_game_info])

In [9]:
all_widgets = widgets.VBox([models_widgets,game_widgets])

In [10]:
all_widgets